Apache Spark practice


#### [학습목표]

- Apache Spark 기초실습


#### [학습기록]


- SQL은 쿼리를 수정하거나 개선할때 어렵고, 쓸때없이 길어질 수 있지만 반면에 spark dataframe는 프로그래밍 코드로 되어 있기 때문에 이 코드들을 재활용하거나 조합하는등 조금 더 유연하다고 할 수 있음


- 데이터 분석 시 액셀을 많이 활용하므로 자유롭게 다룰 수 있어야 한다. 데이터의 용량이 크지 않은경우는 사실상 액셀로 모든 분석을 다 할 수 있다고 말할 수 있다. 20 ~ 30MB, 로우는 1만개 ~ 10만개까지는 액셀이 커버 가능하기 때문에 액셀을 쓰고, 수백메가바이트 정도의 데이터는 로컬에서 파이썬 판다스나 스파크 데이터 프레임으로 데이터를 돌리게되고, 그 이상이만 여러대로 처리한다.


- 예를 들어서 왕좌의 게임 데이터에서 어느 가문이 가장 많이 등장하는가를 산출하고자 할 때는 이런게 결국에는 집계하는 것이다. 어떠한 기준으로 데이터를 뭉쳐서 산출하자는 것인데 SQL로 치면 group by sum 등을 하는 것이다. 집계기능은 액셀에서는 피벗테이블이라는 강력한 도구를 제공한다.

#### [ 왕좌의 게임 등장인물 데이터를 활용한 아파치 스파크 실습 ]

- 데이터 관련 URL : https://www.kaggle.com/mylesoneill/game-of-thrones/data

In [1]:
# 최초 spark 구동

import findspark
findspark.init(r"C:\Users\minman\Desktop\css\spark\spark-2.4.0-bin-hadoop2.7")
# 리눅스에서는 절대경로를 넣어주면 된다

import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

data_path = r"C:\Users\minman\Downloads\game-of-thrones\character-deaths.csv"
char_death_df = spark.read.option("header", "true").csv(data_path)
char_death_df.show()

+--------------------+---------------+----------+-------------+-------------+------------------+------+--------+---+---+---+---+---+
|                Name|    Allegiances|Death Year|Book of Death|Death Chapter|Book Intro Chapter|Gender|Nobility|GoT|CoK|SoS|FfC|DwD|
+--------------------+---------------+----------+-------------+-------------+------------------+------+--------+---+---+---+---+---+
|      Addam Marbrand|      Lannister|      null|         null|         null|                56|     1|       1|  1|  1|  1|  1|  0|
|Aegon Frey (Jingl...|           None|       299|            3|           51|                49|     1|       1|  0|  0|  1|  0|  0|
|     Aegon Targaryen|House Targaryen|      null|         null|         null|                 5|     1|       1|  0|  0|  0|  0|  1|
|       Adrack Humble|  House Greyjoy|       300|            5|           20|                20|     1|       1|  0|  0|  0|  0|  1|
|      Aemon Costayne|      Lannister|      null|         null|      

#### 1) 가문별 사람 수를 group by 함수를 이용해서 출력해보기

In [2]:
char_death_df.groupBy("Allegiances").count().show()

+---------------+-----+
|    Allegiances|count|
+---------------+-----+
|    House Stark|   35|
|           None|  253|
|House Baratheon|    8|
|    House Tully|    8|
|  House Martell|   12|
|        Greyjoy|   51|
|          Arryn|   23|
|   House Tyrell|   11|
|  House Greyjoy|   24|
|House Targaryen|   19|
|      Baratheon|   56|
|      Lannister|   81|
|      Targaryen|   17|
|          Tully|   22|
|        Martell|   25|
|         Tyrell|   15|
|    House Arryn|    7|
|  Night's Watch|  116|
|          Stark|   73|
|       Wildling|   40|
+---------------+-----+
only showing top 20 rows



In [3]:
from pyspark.sql.functions import col

# 아래 세줄의 코드는 모두 서로같은 표현이다.

# char_death_df.groupBy(char_death_df.Allegiances).count().show()
# char_death_df.groupBy(col("Allegiances")).count().show()
char_death_df.groupBy(char_death_df["Allegiances"]).count().show()

+---------------+-----+
|    Allegiances|count|
+---------------+-----+
|    House Stark|   35|
|           None|  253|
|House Baratheon|    8|
|    House Tully|    8|
|  House Martell|   12|
|        Greyjoy|   51|
|          Arryn|   23|
|   House Tyrell|   11|
|  House Greyjoy|   24|
|House Targaryen|   19|
|      Baratheon|   56|
|      Lannister|   81|
|      Targaryen|   17|
|          Tully|   22|
|        Martell|   25|
|         Tyrell|   15|
|    House Arryn|    7|
|  Night's Watch|  116|
|          Stark|   73|
|       Wildling|   40|
+---------------+-----+
only showing top 20 rows



In [4]:
# 정렬까지해서 가문별 사람 수 출력, 변수에 할당
df1 = char_death_df.groupBy("Allegiances").count()
df1.orderBy(col("count").desc()).show()

+---------------+-----+
|    Allegiances|count|
+---------------+-----+
|           None|  253|
|  Night's Watch|  116|
|      Lannister|   81|
|          Stark|   73|
|      Baratheon|   56|
|        Greyjoy|   51|
|       Wildling|   40|
|    House Stark|   35|
|        Martell|   25|
|  House Greyjoy|   24|
|          Arryn|   23|
|          Tully|   22|
|House Lannister|   21|
|House Targaryen|   19|
|      Targaryen|   17|
|         Tyrell|   15|
|  House Martell|   12|
|   House Tyrell|   11|
|    House Tully|    8|
|House Baratheon|    8|
+---------------+-----+
only showing top 20 rows



#### 2) join 함수를 이용한 가문별 사망률 구하기

In [7]:
# withColumn과 isNull함수를 이용한 is_alive 컬럼 추가하기
char_death_df2 = char_death_df.withColumn("is_alive", col("Death Year").isNull())
char_death_df2.show()

+--------------------+---------------+----------+-------------+-------------+------------------+------+--------+---+---+---+---+---+--------+
|                Name|    Allegiances|Death Year|Book of Death|Death Chapter|Book Intro Chapter|Gender|Nobility|GoT|CoK|SoS|FfC|DwD|is_alive|
+--------------------+---------------+----------+-------------+-------------+------------------+------+--------+---+---+---+---+---+--------+
|      Addam Marbrand|      Lannister|      null|         null|         null|                56|     1|       1|  1|  1|  1|  1|  0|    true|
|Aegon Frey (Jingl...|           None|       299|            3|           51|                49|     1|       1|  0|  0|  1|  0|  0|   false|
|     Aegon Targaryen|House Targaryen|      null|         null|         null|                 5|     1|       1|  0|  0|  0|  0|  1|    true|
|       Adrack Humble|  House Greyjoy|       300|            5|           20|                20|     1|       1|  0|  0|  0|  0|  1|   false|
|     

In [8]:
# "Allegiances", "is_alive" 컬럼을 이용한 result_df 피벗테이블 만들기
result_df = char_death_df2.groupBy("Allegiances", "is_alive") \
    .count().orderBy("Allegiances", "is_alive")

result_df.show()

+---------------+--------+-----+
|    Allegiances|is_alive|count|
+---------------+--------+-----+
|          Arryn|   false|    3|
|          Arryn|    true|   20|
|      Baratheon|   false|   20|
|      Baratheon|    true|   36|
|        Greyjoy|   false|    8|
|        Greyjoy|    true|   43|
|    House Arryn|   false|    2|
|    House Arryn|    true|    5|
|House Baratheon|   false|    4|
|House Baratheon|    true|    4|
|  House Greyjoy|   false|   14|
|  House Greyjoy|    true|   10|
|House Lannister|   false|   12|
|House Lannister|    true|    9|
|  House Martell|   false|    1|
|  House Martell|    true|   11|
|    House Stark|   false|   19|
|    House Stark|    true|   16|
|House Targaryen|   false|    4|
|House Targaryen|    true|   15|
+---------------+--------+-----+
only showing top 20 rows



In [9]:
## result 데이터프레임을 두개의 데이터 프레임으로 쪼갠다. is_alive가 true인지 false인지를 기준으로..

df1 = result_df.filter(col("is_alive") == True) \
    .withColumnRenamed("count", "alive_count")

df1.show()

+---------------+--------+-----------+
|    Allegiances|is_alive|alive_count|
+---------------+--------+-----------+
|          Arryn|    true|         20|
|      Baratheon|    true|         36|
|        Greyjoy|    true|         43|
|    House Arryn|    true|          5|
|House Baratheon|    true|          4|
|  House Greyjoy|    true|         10|
|House Lannister|    true|          9|
|  House Martell|    true|         11|
|    House Stark|    true|         16|
|House Targaryen|    true|         15|
|    House Tully|    true|          3|
|   House Tyrell|    true|         10|
|      Lannister|    true|         63|
|        Martell|    true|         23|
|  Night's Watch|    true|         60|
|           None|    true|        177|
|          Stark|    true|         46|
|      Targaryen|    true|         12|
|          Tully|    true|         18|
|         Tyrell|    true|         14|
+---------------+--------+-----------+
only showing top 20 rows



In [10]:
df2 = result_df.filter(col("is_alive") == False) \
    .withColumnRenamed("Allegiances", "Allegiances1") \
    .withColumnRenamed("count", "death_count")

df2.show()

+---------------+--------+-----------+
|   Allegiances1|is_alive|death_count|
+---------------+--------+-----------+
|          Arryn|   false|          3|
|      Baratheon|   false|         20|
|        Greyjoy|   false|          8|
|    House Arryn|   false|          2|
|House Baratheon|   false|          4|
|  House Greyjoy|   false|         14|
|House Lannister|   false|         12|
|  House Martell|   false|          1|
|    House Stark|   false|         19|
|House Targaryen|   false|          4|
|    House Tully|   false|          5|
|   House Tyrell|   false|          1|
|      Lannister|   false|         18|
|        Martell|   false|          2|
|  Night's Watch|   false|         56|
|           None|   false|         76|
|          Stark|   false|         27|
|      Targaryen|   false|          5|
|          Tully|   false|          4|
|         Tyrell|   false|          1|
+---------------+--------+-----------+
only showing top 20 rows



In [11]:
df1.join(df2, df1["Allegiances"] == df2["Allegiances1"]).show()

+---------------+--------+-----------+---------------+--------+-----------+
|    Allegiances|is_alive|alive_count|   Allegiances1|is_alive|death_count|
+---------------+--------+-----------+---------------+--------+-----------+
|          Arryn|    true|         20|          Arryn|   false|          3|
|      Baratheon|    true|         36|      Baratheon|   false|         20|
|        Greyjoy|    true|         43|        Greyjoy|   false|          8|
|    House Arryn|    true|          5|    House Arryn|   false|          2|
|House Baratheon|    true|          4|House Baratheon|   false|          4|
|  House Greyjoy|    true|         10|  House Greyjoy|   false|         14|
|House Lannister|    true|          9|House Lannister|   false|         12|
|  House Martell|    true|         11|  House Martell|   false|          1|
|    House Stark|    true|         16|    House Stark|   false|         19|
|House Targaryen|    true|         15|House Targaryen|   false|          4|
|    House T

** join 관련 참고자료

<img src="1.png" width="700" />

In [12]:
# join을 이용한 결과 데이터 프레임 산출
result = df1.join(df2, df1["Allegiances"] == df2["Allegiances1"]) \
    .select("Allegiances", "alive_count", "death_count", \
       (col("alive_count") / (col("alive_count") + col("death_count"))).alias("alive_rate"))

result.show()

+---------------+-----------+-----------+-------------------+
|    Allegiances|alive_count|death_count|         alive_rate|
+---------------+-----------+-----------+-------------------+
|          Arryn|         20|          3| 0.8695652173913043|
|      Baratheon|         36|         20| 0.6428571428571429|
|        Greyjoy|         43|          8| 0.8431372549019608|
|    House Arryn|          5|          2| 0.7142857142857143|
|House Baratheon|          4|          4|                0.5|
|  House Greyjoy|         10|         14| 0.4166666666666667|
|House Lannister|          9|         12|0.42857142857142855|
|  House Martell|         11|          1| 0.9166666666666666|
|    House Stark|         16|         19|0.45714285714285713|
|House Targaryen|         15|          4| 0.7894736842105263|
|    House Tully|          3|          5|              0.375|
|   House Tyrell|         10|          1| 0.9090909090909091|
|      Lannister|         63|         18| 0.7777777777777778|
|       

#### 3) 스파크 데이터 프레임을 판다스 데이터 프레임으로 변환해보기 

In [6]:
result = df1.join(df2, df1["Allegiances"] == df2["Allegiances1"]) \
    .select("Allegiances", "alive_count", "death_count", \
       (col("alive_count") / (col("alive_count") + col("death_count"))).alias("alive_rate"))

result.toPandas()

,Allegiances,alive_count,death_count,alive_rate
0,Arryn,20,3,0.869565
1,Baratheon,36,20,0.642857
2,Greyjoy,43,8,0.843137
3,House Arryn,5,2,0.714286
4,House Baratheon,4,4,0.500000
5,House Greyjoy,10,14,0.416667
6,House Lannister,9,12,0.428571
7,House Martell,11,1,0.916667
8,House Stark,16,19,0.457143
9,House Targaryen,15,4,0.789474


#### 4) window function을 이용하여 매년 죽은인물들의 가문 Top5를 뽑아보기

- window function이란 행과 행간의 관계를 쉽게 정의하기 위해 만든 함수다.

- 집계한다는 개념보다는 특정 컬럼에서 특정 데이터의 하나하나의 종합적인 정보를 알기위해 많이 쓴다.

In [14]:
from pyspark.sql.window import Window
from pyspark.sql.functions import * 

result = char_death_df.groupBy('Death Year', 'Allegiances').count().orderBy(col('Death Year'))
result.show()

+----------+---------------+-----+
|Death Year|    Allegiances|count|
+----------+---------------+-----+
|      null|       Wildling|   17|
|      null|  Night's Watch|   60|
|      null|          Tully|   18|
|      null|        Martell|   23|
|      null|    House Arryn|    5|
|      null|    House Stark|   16|
|      null|          Stark|   46|
|      null|      Baratheon|   36|
|      null|      Targaryen|   12|
|      null|         Tyrell|   14|
|      null|      Lannister|   63|
|      null|  House Martell|   11|
|      null|    House Tully|    3|
|      null|           None|  177|
|      null|House Targaryen|   15|
|      null|House Baratheon|    4|
|      null|        Greyjoy|   43|
|      null|  House Greyjoy|   10|
|      null|          Arryn|   20|
|      null|   House Tyrell|   10|
+----------+---------------+-----+
only showing top 20 rows



In [16]:
window = Window.partitionBy("Death Year").orderBy(col("count").desc())

# 윈도우네는 파티셔닝과 오더링 두개가 들어가는데 파티셔닝이 그룹바이랑 비슷하다고 생각하면 되는 것이고, 오더링은 로우 넘버를 정해서
# 무슨순서로 줄을 세운다음 로우넘버를 셀건지 라는 것이다.

result.select('*', row_number().over(window).alias('n')).filter(col('n') <= 5).show()

# row_number는 집계기준을 어떻게 잡을것인가 설정해주는 것이고, 그것을 .over(window) 처럼 윈도우를 기준으로 하겠다는 것이다.

+----------+-------------+-----+---+
|Death Year|  Allegiances|count|  n|
+----------+-------------+-----+---+
|      null|         None|  177|  1|
|      null|    Lannister|   63|  2|
|      null|Night's Watch|   60|  3|
|      null|        Stark|   46|  4|
|      null|      Greyjoy|   43|  5|
|       297|Night's Watch|    3|  1|
|       298|        Stark|   10|  1|
|       298|  House Stark|    9|  2|
|       298|         None|    7|  3|
|       298|    Targaryen|    4|  4|
|       298|     Wildling|    3|  5|
|       299|         None|   35|  1|
|       299|Night's Watch|   33|  2|
|       299|        Stark|   17|  3|
|       299|    Baratheon|   16|  4|
|       299|House Greyjoy|   11|  5|
|       300|         None|   34|  1|
|       300|Night's Watch|   18|  2|
|       300|     Wildling|   14|  3|
|       300|    Lannister|    7|  4|
+----------+-------------+-----+---+
only showing top 20 rows



#### 5) UDF(User Defined Function) 생성 및 사용해보기

- 파이썬 함수를 만들어서 판다스 데이터프레임에 적용하는 것과 비슷한것으로 마찬가지로 파이썬 함수를 만들어서 스파크 데이터프레임에 적용이 가능하다.

- 간단한 함수는 람다를 써도 되고, 만드려는 함수가 복잡해지면 UDF가 유용한 기능이다.

In [73]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

def genderFunc(gender):
    if gender == 1:
        return "Female" 
    else:  
        return "Male"

genderStringUDF = udf(genderFunc, StringType())

char_death_df.withColumn("Gender2", genderStringUDF(col("Gender"))).show()

+--------------------+---------------+----------+-------------+-------------+------------------+------+--------+---+---+---+---+---+-------+
|                Name|    Allegiances|Death Year|Book of Death|Death Chapter|Book Intro Chapter|Gender|Nobility|GoT|CoK|SoS|FfC|DwD|Gender2|
+--------------------+---------------+----------+-------------+-------------+------------------+------+--------+---+---+---+---+---+-------+
|      Addam Marbrand|      Lannister|      null|         null|         null|                56|     1|       1|  1|  1|  1|  1|  0|   Male|
|Aegon Frey (Jingl...|           None|       299|            3|           51|                49|     1|       1|  0|  0|  1|  0|  0|   Male|
|     Aegon Targaryen|House Targaryen|      null|         null|         null|                 5|     1|       1|  0|  0|  0|  0|  1|   Male|
|       Adrack Humble|  House Greyjoy|       300|            5|           20|                20|     1|       1|  0|  0|  0|  0|  1|   Male|
|      Aemon 

#### 6) 출력한 데이터 프레임을 파일로 저장하기

In [80]:
# 기본적으로 스파크는 하둡분산시스템이기 때문에 파일도 분산저장된다. 
# 그래서 설정을 잘 걸어줘야 원하는 형태의 파일로 저장할 수 있다.

result.repartition(1).write.mode("overwrite").csv("result.csv", header=True, sep='\t')

In [ ]:
# 또한 jdbc writer를 쓰면 데이터베이스에 데이터를 쉽게 저장할 수 있다.
# json이나 Parquet, 오알씨 같은 빅데이터 포멧으로도 저장할 수 있다. csv는 Parquet로 변환해서 저장하면 된다. 

result.write.jdbc(url="jdbc://JDBC_ADDRESS/database_name", "table_name")